## Actividad 2: Entrenamiento de modelo para regresion

En el presente notebook que realiza el paso a paso para entrenamiento de modelo de aprendizaje supervisado que resuleve el problema de regresión para predecir el valor del alquiler de un inmueble según Airbnb

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

Clase para extraer columnas de un DataFrame de pandas y si es necesario, aplicarle un encoder

In [2]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns: list, transformer=None):
        self.columns = columns
        self.transformer = transformer

    def fit(self, X: dict, y=None, **fit_params):
        """
        Fit vectorizer
        """
        X = dict((k, X[k]) for k in self.columns)
        data = np.array(list(X.values()))
        if self.transformer is not None:
            self.transformer.fit(data.transpose(), y, **fit_params)

        return self

    def transform(self, X: dict, **transform_params):
        """
        Return column(s)
        """
        X = dict((k, X[k]) for k in self.columns)
        data = np.array(list(X.values()))
        if self.transformer is not None:
            tfs = self.transformer.transform(
                data.transpose(),
                **transform_params
            )
            return tfs
        else:
            return data.transpose()

In [3]:
pd.options.display.max_columns = 100

## Carga de dataset limpio

In [4]:
path = 'https://raw.githubusercontent.com/eyberthrojas/Actividad_2/main/dataset_cleanned_airbnb_seatle.csv'
df = pd.read_csv(path, on_bad_lines='skip')

In [ ]:
df.sample(3)

Primero que todo eliminamos las columnas que tienen correlación nula con la columna "price" segun el proiceso de ETL realizado en la actividad 1.

In [6]:
df.drop(labels=['requires_license', 'has_availability', 'square_feet'], axis=1, inplace=True)

Vamos a entrenar un modelo de regresión utilizando la arquitectura randon_orest_regressor, pero antes de deben eliminar o reemplazar por algun valor los NAN.

- primero eliminado las filas que contengan algun valor NAN

In [ ]:
df.shape

In [ ]:
df_nan = df.dropna().copy()
df_nan.shape

Como vemos, se eliminan casi la tercera parte de los casos, podemos estar perdiendo mucha información, vamos a entrenar el modelo:

In [9]:
X = df_nan.drop(['price'],axis = 1)
y = df_nan['price']

In [10]:
categoricas =list(X.select_dtypes(include=['object','category']).columns)
numericas = list(X.select_dtypes(include='number').columns)

In [ ]:
# Variables categoricas
categoricas

In [ ]:
# Variables numéricas
numericas

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state=42)

In [14]:
# Union de caracteristicas categoricas y numéricas codificadas
features = FeatureUnion([
    ('categorical_features', FeatureExtractor(columns=categoricas, transformer=OneHotEncoder(handle_unknown='ignore'))),
    ('numeric_features', FeatureExtractor(columns=numericas, transformer=StandardScaler()))
])

In [18]:
# Construcción de pipeline del modelo
model = Pipeline([
    ('features', features),
    ('model', RandomForestRegressor(n_estimators=100, criterion = 'friedman_mse'))
])

In [ ]:
model.fit(X_train, y_train)

In [20]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

La métrica más común para evaluar un modelo de regresión es el r2_score, es una medida de la calidad de ajuste de un modelo de regresión. Es un número entre 0 y 1 que indica la proporción de la varianza en la variable de salida que es explicada por el modelo.

Un R2 score de 1 indica que el modelo ajusta perfectamente los datos, mientras que un R2 score de 0 indica que el modelo no explica nada de la variabilidad en la variable de salida.

In [ ]:
print(f'r2_score con datos de entrenamiento: {r2_score(y_train, train_preds)}')
print(f'r2_score con datos de test: {r2_score(y_test, test_preds)}')

Ahora probemos no eliminando los NAN por el valor promedio de la columna:

In [ ]:
# Primero eliminamos los NAN de la variables categoricas
df_mean = df.dropna(subset=categoricas+['price'], axis=0)
# Luego, rellenamos los NaN con el valor promedio de la columna
for numerica in numericas:
  df_mean[numerica] = df_mean[numerica].fillna(df_mean[numerica].mean())

In [ ]:
df_mean.head()

In [ ]:
df_mean.shape

In [25]:
X = df_mean.drop(['price'],axis = 1)
y = df_mean['price']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state=42)

In [29]:
model_2 = Pipeline([
    ('features', features),
    ('model', RandomForestRegressor(n_estimators=100, criterion = 'friedman_mse'))
])

In [ ]:
model_2.fit(X_train, y_train)

In [31]:
train_preds = model_2.predict(X_train)
test_preds = model_2.predict(X_test)

In [ ]:
print(f'r2_score con datos de entrenamiento: {r2_score(y_train, train_preds)}')
print(f'r2_score con datos de test: {r2_score(y_test, test_preds)}')